In [1]:
from fastai.vision.all import *

/workspace/fastai/fastai/callback/hook.py:190: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if np is not '':


In [2]:
torch.backends.cudnn.benchmark = True

In [3]:
set_seed(999, reproducible=True)

In [18]:
set_seed??

In [14]:
no_random??

Configuration:

In [4]:
gpu = 1
woof = 0
lr = 3e-2
size = 128
sqrmom = 0.99
mom = 0.95
eps = 1e-6
epochs = 5
bs = 256/4
mixup = 0.
opt = 'ranger'
arch = 'xse_resnext50'
sh = 0.
sa = 1
sym = 0
beta = 0.
act_fn = 'Mish'
fp16 = 1
pool = 'MaxPool'

In [5]:
def get_dls(size, woof, bs, sh=0., workers=None):
    if size<=224: path = URLs.IMAGEWOOF_320 if woof else URLs.IMAGENETTE_320
    else        : path = URLs.IMAGEWOOF     if woof else URLs.IMAGENETTE
    source = untar_data(path)
    if workers is None: workers = min(8, num_cpus())
    batch_tfms = [Normalize.from_stats(*imagenet_stats)]
    if sh: batch_tfms.append(RandomErasing(p=0.3, max_count=3, sh=sh))
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       splitter=GrandparentSplitter(valid_name='val'),
                       get_items=get_image_files, get_y=parent_label,
                       item_tfms=[RandomResizedCrop(size, min_scale=0.35), FlipItem(0.5)],
                       batch_tfms=batch_tfms)
    return dblock.dataloaders(source, path=source, bs=bs, num_workers=workers)
        

In [6]:
if gpu is not None: torch.cuda.set_device(gpu)

In [7]:
if   opt=='adam'  : opt_func = partial(Adam, mom=mom, sqr_mom=sqrmom, eps=eps)
elif opt=='rms'   : opt_func = partial(RMSprop, sqr_mom=sqrmom)
elif opt=='sgd'   : opt_func = partial(SGD, mom=mom)
elif opt=='ranger': opt_func = partial(ranger, mom=mom, sqr_mom=sqrmom, eps=eps, beta=beta)

In [8]:
dls = get_dls(size, woof, bs, sh=sh)

In [9]:
m,act_fn,pool = [globals()[o] for o in (arch,act_fn,pool)]

In [10]:
learn = Learner(dls, m(n_out=10, act_cls=act_fn, sa=sa, sym=sym, pool=pool), opt_func=opt_func, \
                metrics=[accuracy,top_k_accuracy], loss_func=LabelSmoothingCrossEntropy())

In [19]:
NativeMixedPrecision??

In [11]:
if fp16: learn = learn.to_native_fp16()
cbs = MixUp(mixup) if mixup else []

In [12]:
#learn.lr_find()

In [13]:
learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)

AttributeError: 'Learner' object has no attribute 'pbar'

In [ ]:
learn.recorder.plot_loss()